[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/06-langchain-agents.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/06-langchain-agents.ipynb)

#### [LangChain Handbook](https://pinecone.io/learn/langchain)

# Agents 🤖

Agents are like "tools" for LLMs. They allow a LLM to access Google search, perform complex calculations with Python, and even make SQL queries.

In this notebook we'll explore agents and how to use them in LangChain.

We'll start by installing the prerequisite libraries that we'll be using in this example.

In [1]:
!pip install -qU \
  langchain==0.3.25 \
  langchain-openai==0.3.22 \
  langchain-experimental==0.3.4 \
  numexpr==2.11.0 \
  google-search-results==2.4.2 \
  wikipedia==1.4.0 \
  sqlalchemy==2.0.41 \

ERROR: Directory '\\' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use for the whole notebook, just input your openai api key when prompted.

In [2]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
    or getpass("Enter your OpenAI API key: ")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-4.1-mini",
    temperature=0.0,
)

As we did before, we will be counting our tokens in each call.

In [4]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent.invoke(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

With all of that set up, let's jump into **Agents**.

## What is an agent?

**Definition**: The key behind agents is giving LLM's the possibility of using tools in their workflow. This is where langchain departs from the popular chatgpt implementation and we can start to get a glimpse of what it offers us as builders. Until now, we covered several building blocks in isolation. Let's see them come to life.

The official definition of agents is the following:


> Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

In this edition we will cover what we may call 'generic' agents which really able to perform many meta tasks. There are other more specific agents that are tuned for different tasks (called 'agent-toolkits'), but we will cover those in a future edition.

## Create database

We will use the agents to interact with a small sample database of stocks. We will not dive into the details because this is just a dummy tool we will build for illustrative purposes. Let's create it.

In [5]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [6]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [7]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [8]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [9]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [10]:
for obs in observations:
    insert_obs(obs)

We are installing the `langchain_experimental` library here, since the `SQLDatabaseChain` is located there. This might be changed in the future and moved into official `langchain` library.

In [11]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)

## Agent types

In this section we will review several agents and see how they 'think' and what they can do.

Using one of langchain's pre-built agents involves three variables:
* defining the tools or the toolkit
* defining the llm
* defining the agent type

This is all really easy to do in langchain, as we will see in the following example.

### Agent type #1: Zero Shot React

In this first example we will use slightly different type of agent - SQL Agent which can be instantiated with it's own method `create_sql_agent`. Other agents will be instantiated in more generic way as we will see below in other examples.
<br><br>
This method uses *toolkit* instead of simple list of `tools`. You can read more about them in the [documentation](https://python.langchain.com/docs/modules/agents/toolkits/). For this use case, we will use `SQLDatabaseToolkit`.

As the name suggests, we will use this agent to perform 'zero shot' tasks on the input. That means that we will not have several, interdependent interactions but only one. In other words, this agent will have no memory.

Now we are ready to initialize the agent! We will use `verbose` in `True` so we can see what is our agent's 'thinking' process.

**Important Note:** *When interacting with agents it is really important to set the `max_iterations` parameters because agents can get stuck in infinite loops that consume plenty of tokens. The default value is 15 to allow for many tools and complex reasoning but for most applications you should keep it much lower.*

In [17]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=5
)

Let's see our newly created agent in action! We will ask it a question that involves a math operation over the stock prices.

In [18]:
result = count_tokens(
    agent_executor,
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " +
    "between the same stock prices in January the 4th?"
)
print(f"Result: {result['output']}")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: stocksThought: There is a table named "stocks" which likely contains stock prices. I should check the schema of the "stocks" table to understand its columns and see how to query the stock prices for 'ABC' and 'XYZ' on January 3rd and January 4th.
Action: sql_db_schema
Action Input: stocks
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2023-01-01
2	ABC	208.0	2023-01-02
3	ABC	232.0	2023-01-03
*/Thought: I need to get the prices for 'ABC' and 'XYZ' on January 3rd and January 4th. Then I will calculate the ratio of ABC to XYZ on each date and multiply these two ratios.

Action: sql_db_query_checker
Action Input: 
SELECT stock_ticker, price, date FROM stocks WHERE stock_ticker IN ('ABC', 'XYZ') AND date IN ('2023-01-03', '2023-0

As always, let's see what the prompt is here:

In [22]:
print(agent_executor.agent.runnable.steps[1].template)

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't k

The question we must ask ourselves here is: how are agents different than chains?

If we look at the agent's logic and the prompt we have just printed we will see some clear differences. First, we have the tools which are included in the prompt. Second we have a thought process which was before was immediate in chains but now involves a 'thought', 'action', 'action input', 'observation' sequence. What is this all about?

Suffice it to say for now that **the LLM now has the ability to 'reason' on how to best use tools** to solve our query and can combine them in intelligent ways with just a brief description of each of them. If you want to learn more about this paradigm (MRKL) in detail, please refer to [this](https://arxiv.org/pdf/2205.00445.pdf) paper.

Finally, let's pay attention to the 'agent_scratchpad'. What is that? Well, that is where we will be appending every thought or action that the agent has already performed. In this way, at each point in time, the agent will know what it has found out and will be able to continue its thought process. In other words, after using a tool it adds its thoughts and observations to the scratchpad and picks up from there.

### Agent type #2: Conversational React

The zero shot agent is really interesting but, as we said before, it has no memory. What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? For that we have the conversational react agent.

We will use the math tool in this example and load it as below:

In [23]:
# Use the modern tool-based approach instead of deprecated LLMMathChain
from langchain_core.tools import tool
import math
import numexpr

@tool
def calculator(expression: str) -> str:
    """Calculate expression using Python's numexpr library.
    
    Expression should be a single line mathematical expression
    that solves the problem.
    
    Examples:
        "37593 * 67" for "37593 times 67"
        "37593**(1/5)" for "37593^(1/5)"
        "10000 * (1 + 0.08)**5" for compound interest
    """
    local_dict = {"pi": math.pi, "e": math.e}
    return str(
        numexpr.evaluate(
            expression.strip(),
            global_dict={},  # restrict access to globals
            local_dict=local_dict,  # add common mathematical functions
        )
    )

tools = [calculator]

The memory type being used here is a simple buffer memory to allow us to remember previous steps in the reasoning chain. For more information on memory, please refer to the 3rd chapter of this series.

In [24]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

C:\Users\Siraj\AppData\Local\Temp\ipykernel_94028\4074810399.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [111]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.base import RunnableSerializable
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
import json

# Add a final_answer tool as recommended in the guide
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

# Add tools
tools = [final_answer, calculator]

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user. "
        "MOST IMPORTANT RULE: Always check the conversation history first. "
        "If the history contains a calculation result you need, use that result. "
        "Do not recalculate anything that has already been calculated in the conversation. "
        "This is more efficient and avoids redundant work."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# define the agent runnable with tool_choice="auto" instead of "any"
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")  # Allow LLM to choose whether to use tools
)

# create tool name to function mapping as per guide
name2tool = {tool.name: tool.func for tool in tools}

class CustomAgentExecutor:
    def __init__(self, max_iterations: int = 5):
        self.max_iterations = max_iterations
        self.chat_history = []  # Simple list to store conversation history
        self.agent = agent

    def invoke(self, input: str) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []
        
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            tool_call = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            
            # add initial tool call to scratchpad
            agent_scratchpad.append(tool_call)
            
            # Handle ALL tool calls, not just the first one
            if tool_call.tool_calls:
                for tool_call_obj in tool_call.tool_calls:
                    tool_name = tool_call_obj["name"]
                    tool_args = tool_call_obj["args"]
                    tool_call_id = tool_call_obj["id"]
                    
                    # execute the tool
                    tool_out = name2tool[tool_name](**tool_args)
                    
                    # add the tool output to the agent scratchpad
                    tool_exec = ToolMessage(
                        content=f"{tool_out}",
                        tool_call_id=tool_call_id
                    )
                    agent_scratchpad.append(tool_exec)
                    
                    # add a print so we can see intermediate steps
                    print(f"{count}: {tool_name}({tool_args}) -> {tool_out}")
                
                count += 1
                
                # Check if any tool call is the final answer tool
                if any(tc["name"] == "final_answer" for tc in tool_call.tool_calls):
                    # Get the final answer from the final_answer tool
                    final_tool_call = next(tc for tc in tool_call.tool_calls if tc["name"] == "final_answer")
                    final_answer = final_tool_call["args"]["answer"]
                    break
            else:
                # no tool call, we have a final answer
                final_answer = tool_call.content
                break
        
        # Add to conversation history ONLY the human input and final AI response
        # This preserves memory without corrupting it with tool calls
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_answer)
        ])
        
        # return the final answer in dict form
        return {"output": final_answer}

# Initialize the custom agent executor
conversational_agent = CustomAgentExecutor()

In [112]:
# First question
result = conversational_agent.invoke("What is 10000 * (1 + 0.08)**5?")
print(f"Result: {result['output']}")

0: calculator({'expression': '10000 * (1 + 0.08)**5'}) -> 14693.280768000006
1: final_answer({'answer': 'The value of 10000 * (1 + 0.08)**5 is approximately 14693.28.', 'tools_used': ['functions.calculator']}) -> {'answer': 'The value of 10000 * (1 + 0.08)**5 is approximately 14693.28.', 'tools_used': ['functions.calculator']}
Result: The value of 10000 * (1 + 0.08)**5 is approximately 14693.28.


As we can see below, the prompt is similar but it includes a great prelude of instructions that make it an effective assistant as well + a spot for including the chat history from the memory component:

In [113]:
# To see the actual template content, we need to format it with sample values
print("Prompt Template Structure:")
print("=" * 50)
print("System message:", prompt.messages[0].prompt.template)
print("Human message template:", prompt.messages[2].prompt.template)
print("Variables:", prompt.input_variables)
print("=" * 50)

Prompt Template Structure:
System message: You're a helpful assistant. When answering a user's question you should first use one of the tools provided. After using a tool the tool output will be provided in the 'scratchpad' below. If you have an answer in the scratchpad you should not use any more tools and instead answer directly to the user. MOST IMPORTANT RULE: Always check the conversation history first. If the history contains a calculation result you need, use that result. Do not recalculate anything that has already been calculated in the conversation. This is more efficient and avoids redundant work.
Human message template: {input}
Variables: ['agent_scratchpad', 'chat_history', 'input']


Let's see what happens if we try to answer the question that is related to the previous one:

In [114]:
result = conversational_agent.invoke(
    "If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?"
)
print(f"Result: {result['output']}")

0: calculator({'expression': '15000 * (1 + 0.08)**5'}) -> 22039.92115200001
0: calculator({'expression': '10000 * (1 + 0.08)**5'}) -> 14693.280768000006
1: calculator({'expression': '22039.92115200001 - 14693.280768000006'}) -> 7346.640384000004
2: final_answer({'answer': 'If we start with $15,000 instead of $10,000 and follow the same 8% annual growth for 5 years with compound interest, we would have $7,346.64 more compared to the previous scenario.', 'tools_used': ['multi_tool_use.parallel', 'functions.calculator', 'functions.calculator', 'functions.final_answer']}) -> {'answer': 'If we start with $15,000 instead of $10,000 and follow the same 8% annual growth for 5 years with compound interest, we would have $7,346.64 more compared to the previous scenario.', 'tools_used': ['multi_tool_use.parallel', 'functions.calculator', 'functions.calculator', 'functions.final_answer']}
Result: If we start with $15,000 instead of $10,000 and follow the same 8% annual growth for 5 years with comp

In [115]:
# Debug: Let's see what the actual formatted prompt looks like
print("Formatted prompt messages:")
print("=" * 50)
try:
    formatted_messages = prompt.format_messages(
        input="If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?",
        chat_history=conversational_agent.chat_history,
        agent_scratchpad=[]
    )
    
    for i, msg in enumerate(formatted_messages):
        print(f"{i}: {type(msg).__name__}")
        print(f"   Content: {msg.content}")
        print()
        
except Exception as e:
    print(f"Error formatting prompt: {e}")
print("=" * 50)

Formatted prompt messages:
0: SystemMessage
   Content: You're a helpful assistant. When answering a user's question you should first use one of the tools provided. After using a tool the tool output will be provided in the 'scratchpad' below. If you have an answer in the scratchpad you should not use any more tools and instead answer directly to the user. MOST IMPORTANT RULE: Always check the conversation history first. If the history contains a calculation result you need, use that result. Do not recalculate anything that has already been calculated in the conversation. This is more efficient and avoids redundant work.

1: HumanMessage
   Content: What is 10000 * (1 + 0.08)**5?

2: AIMessage
   Content: The value of 10000 * (1 + 0.08)**5 is approximately 14693.28.

3: HumanMessage
   Content: If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?

4: AIMessage
   Content:

### Agent type #3: React Docstore

This type of agent is similar to the ones we have seen so far but it includes the interaction with a docstore. It will have two and only two tools at its disposal: 'Search' and 'Lookup'.

With 'Search' it will bring up a relevant article and with 'Lookup' the agent will find the right piece of information in the article. This is probably easiest to see in an example:

In [84]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool

@tool
def Search(query: str) -> str:
    """Search Wikipedia for information about a topic."""
    try:
        wiki = WikipediaAPIWrapper()
        return wiki.run(query)
    except Exception as e:
        return f"Error searching Wikipedia: {e}"

@tool
def Lookup(term: str) -> str:
    """Look up a specific term or phrase in Wikipedia."""
    try:
        wiki = WikipediaAPIWrapper()
        return wiki.run(term)
    except Exception as e:
        return f"Error looking up term: {e}"

tools = [Search, Lookup]

In [85]:
# Create a custom agent executor for docstore tools
docstore_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant that can search and lookup information."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

docstore_agent_runnable = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | docstore_prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

docstore_name2tool = {tool.name: tool.func for tool in tools}

class DocstoreAgentExecutor:
    def __init__(self, max_iterations: int = 3):
        self.max_iterations = max_iterations
        self.agent = docstore_agent_runnable

    def invoke(self, input: str) -> dict:
        count = 0
        agent_scratchpad = []
        
        while count < self.max_iterations:
            tool_call = self.agent.invoke({
                "input": input,
                "agent_scratchpad": agent_scratchpad
            })
            
            agent_scratchpad.append(tool_call)
            
            if not tool_call.tool_calls:
                final_answer = tool_call.content
                break
                
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = docstore_name2tool[tool_name](**tool_args)
            
            tool_exec = ToolMessage(
                content=f"{tool_out}",
                tool_call_id=tool_call_id
            )
            agent_scratchpad.append(tool_exec)
            
            print(f"{count}: {tool_name}({tool_args}) = {tool_out[:100]}...")
            count += 1
            
        return {"input": input, "output": final_answer}

docstore_agent = DocstoreAgentExecutor()

In [86]:
result = docstore_agent.invoke("What were Archimedes' last words?")
print(f"Result: {result['output']}")

0: Lookup({'term': "Archimedes' last words"}) = Page: Archimedes
Summary: Archimedes of Syracuse ( AR-kih-MEE-deez; c. 287 – c. 212 BC) was an Ancie...
Result: The search did not return specific information about Archimedes' last words. However, historically it is often said that Archimedes' last words were "Do not disturb my circles," which he reportedly said to a Roman soldier who was about to kill him during the siege of Syracuse. This phrase reflects his deep engagement with his mathematical work even at the moment of his death.


We will not print the prompt here because it is too large, but you can see it yourself if you want to (we know how to already).

In short, it contains several examples of the `Question` > `Thought` > `Action` > `Observation` loop, that include the `Search` and `Lookup` tools.

If you want to learn more about this approach [this](https://arxiv.org/pdf/2210.03629.pdf) is the paper for ReAct

### Agent type #4: Self Ask with Search

This is the first-choice agent to use when using LLM's to extract information with a search engine. The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to a final answer.

In [87]:
from langchain.agents import load_tools

# Add SerpAPI key setup
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY") \
    or getpass("Enter your SerpAPI API key: ")

toolbox = load_tools(tool_names=['serpapi'], llm=llm)

# Use modern agent creation pattern
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(
    llm=llm, tools=toolbox, prompt=prompt
)

self_ask_with_search = AgentExecutor(
    agent=agent,
    tools=toolbox,
    verbose=True
)

We will not interact with this agent because for that we would need a serpapi key. However, by checking out the prompt we can see a few examples of how it works:

In [88]:
# The modern agent structure is different - let's inspect it properly
print("Modern Agent Structure:")
print("=" * 50)
print("Agent type:", type(self_ask_with_search.agent).__name__)
print("=" * 50)

# To see the prompt template, we need to access it differently
agent_runnable = self_ask_with_search.agent.runnable
prompt_template = agent_runnable.steps[1]

print("Prompt Template Structure:")
print("=" * 50)
print("System message:", prompt_template.messages[0].prompt.template)
print("Human message template:", prompt_template.messages[1].prompt.template)
print("Required variables:", prompt_template.input_variables)
print("Optional variables:", prompt_template.optional_variables)
print("=" * 50)


Modern Agent Structure:
Agent type: RunnableMultiActionAgent
Prompt Template Structure:
System message: you're a helpful assistant
Human message template: {input}
Required variables: ['input']
Optional variables: ['agent_scratchpad']


Note: The modern create_tool_calling_agent uses a simpler prompt template.
The old 'self-ask-with-search' agent type had detailed examples showing
how to ask follow-up questions, but this modern approach relies more on
the LLM's inherent capabilities and the tool descriptions.

And.. again [here](https://arxiv.org/pdf/2210.03350.pdf) you have the paper to dive deeper!

### Wrapping up

And that all for agents! There's many other things you can do with agents, just to name a few:
* Create your own custom agent
* Use them with many other tools (even custom ones)
* Trace every call an agent makes through a convinient UI interface

Check the how-to [guides](https://langchain.readthedocs.io/en/latest/modules/agents/how_to_guides.html) for more!